# Objective
- Find the average price of the top **4 competitors** across 3 product categories: **Fragrance**, **Makeup**, **Skincare**

# Tasks
1. Reclassify the product categories into the 3 main categories. Coversion table is given beforehand
2. Find the top 4 competitors based on the total number of fragrance, makeup and skincare products
3. Exclude all products that contains the ingredients 'toluene'
4. Convert all price to USD for comparasion and calculation
5. Summarize the average price of 4 competitors as a DataFrame

# Imports

In [60]:
import pandas as pd
import numpy as np

# Load Data

In [61]:
cosmetics_df = pd.read_csv('datasets/cosmetics.csv')
ingredient_df = pd.read_csv('datasets/ingredients.txt', sep = '\t')
category_df = pd.read_csv('datasets/categories.csv')

# Exploratory Data Analysis

In [62]:
cosmetics_df.head()

,id,brand,category,name,size,price,rating,how_to_use,online_only,limited_edition
0,2218774,Acqua Di Parma,Perfume,Blu Mediterraneo MINIATURE Set,5 x 0.16oz/5mL,USD 66,4.0,Suggested Usage:-Fragrance is intensified by t...,1,0
1,2044816,Acqua Di Parma,Cologne,Colonia,0.7 oz/ 20 mL,USD 66,4.5,no instructions,1,0
2,1417567,Acqua Di Parma,Perfume,Arancia di Capri,5 oz/ 148 mL,USD 180,4.5,no instructions,1,0
3,1417617,Acqua Di Parma,Perfume,Mirto di Panarea,2.5 oz/ 74 mL,USD 120,4.5,no instructions,1,0
4,2218766,Acqua Di Parma,Perfume,Colonia Miniature Set,5 x 0.16oz/5mL,USD 72,3.5,Suggested Usage:-Fragrance is intensified by t...,1,0


In [63]:
ingredient_df.head()

,product_id,ingredients
0,2218774,Arancia di Capri Eau de Toilette: Alcohol Dena...
1,2044816,unknown
2,1417567,Alcohol Denat.- Water- Fragrance- Limonene- Li...
3,1417617,unknown
4,2218766,Colonia: Alcohol Denat.- Water- Fragrance- Lim...


In [64]:
category_df.head()

,sub_category,broad_category
0,Perfume,Fragrance
1,Cologne,Fragrance
2,Eye Palettes,Makeup
3,Lipstick,Makeup
4,Highlighter,Makeup


We can see that the `cosmetics_df` can be join with the `ingredient_df` on the `id` and `product_id` columns, and with the `category_df` on the `category` and `sub_category` columns.

As the task does not require some of the `cosmetics_df` columns, we will remove them so that the dimension of the joined DataFrame won't be too big.

In [65]:
cosmetics_df.drop(['size', 'rating', 'how_to_use', 'online_only', 'limited_edition'], axis = 1 , inplace = True)
cosmetics_df.head()

,id,brand,category,name,price
0,2218774,Acqua Di Parma,Perfume,Blu Mediterraneo MINIATURE Set,USD 66
1,2044816,Acqua Di Parma,Cologne,Colonia,USD 66
2,1417567,Acqua Di Parma,Perfume,Arancia di Capri,USD 180
3,1417617,Acqua Di Parma,Perfume,Mirto di Panarea,USD 120
4,2218766,Acqua Di Parma,Perfume,Colonia Miniature Set,USD 72


In [66]:
cosmetics_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8632 entries, 0 to 8631
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        8632 non-null   int64 
 1   brand     8632 non-null   object
 2   category  8632 non-null   object
 3   name      8632 non-null   object
 4   price     8632 non-null   object
dtypes: int64(1), object(4)
memory usage: 337.3+ KB


In [67]:
cosmetics_df['brand'].unique()

array(['Acqua Di Parma', 'acqua di parma', 'ACQUA DI PARMA',
       'Acqua di parma', 'Aerin', 'AERIN', 'aerin', 'Algenist',
       'ALGENIST', 'algenist', 'ALPHA-H', 'Alpha-H', 'alterna haircare',
       'ALTERNA Haircare', 'Alterna haircare', 'ALTERNA HAIRCARE',
       'Alterna Haircare', 'Amika', 'amika', 'AMIKA', 'amorepacific',
       'AMOREPACIFIC', 'Amorepacific', 'Anastasia Beverly Hills',
       'Anastasia beverly hills', 'anastasia beverly hills',
       'ANASTASIA BEVERLY HILLS', 'Anthony', 'ANTHONY', 'Armani Beauty',
       'ARMANI BEAUTY', 'armani beauty', 'Armani beauty',
       'The Art of Shaving', 'the art of shaving', 'The Art Of Shaving',
       'THE ART OF SHAVING', 'Artis', 'artis', 'ARTIS', 'Atelier Cologne',
       'Atelier cologne', 'ATELIER COLOGNE', 'atelier cologne',
       'bareminerals', 'bareMinerals', 'Bareminerals', 'BAREMINERALS',
       'Beautybio', 'beautybio', 'BeautyBio', 'BEAUTYBIO',
       'Beautyblender', 'beautyblender', 'BEAUTYBLENDER', 'BECCA'

Many brand names are inputted into the dataset with different typing. We will convert all brand names to lower case so that it's easier to correctly summarize.

In [68]:
cosmetics_df['brand'] = cosmetics_df['brand'].str.lower()
cosmetics_df['brand'].unique()

array(['acqua di parma', 'aerin', 'algenist', 'alpha-h',
       'alterna haircare', 'amika', 'amorepacific',
       'anastasia beverly hills', 'anthony', 'armani beauty',
       'the art of shaving', 'artis', 'atelier cologne', 'bareminerals',
       'beautybio', 'beautyblender', 'becca', 'belif',
       'benefit cosmetics', 'berdoues', 'better not younger', 'bio ionic',
       'biossance', 'bite beauty', 'bobbi brown', 'boscia', 'briogeo',
       'bumble and bumble', 'burberry', 'bvlgari', 'by rosie jane',
       'capri blue', 'carolina herrera', 'caudalie', 'chanel',
       'charlotte tilbury', 'chloé', 'christian louboutin',
       'christophe robin', 'ciaté london', 'cinema secrets', 'clarins',
       'clarisonic skincare', 'clean reserve', 'clinique', 'coola',
       'cover fx', 'deborah lippmann', 'dermalogica', 'devacurl', 'dior',
       'dolce&gabbana', 'dphue', "dr roebuck's", 'dr. barbara sturm',
       'dr. brandt skincare', 'dr. dennis gross skincare', 'dr. jart+',
       '

The `price` columns consists of both the currenct and the actual price. We will split into 2 columns and cast the `price` columns as float.

In [69]:
cosmetics_df[['currency', 'price']] = cosmetics_df['price'].str.split(expand = True)
cosmetics_df['price'] = cosmetics_df['price'].astype('float')
cosmetics_df.head()

,id,brand,category,name,price,currency
0,2218774,acqua di parma,Perfume,Blu Mediterraneo MINIATURE Set,66.0,USD
1,2044816,acqua di parma,Cologne,Colonia,66.0,USD
2,1417567,acqua di parma,Perfume,Arancia di Capri,180.0,USD
3,1417617,acqua di parma,Perfume,Mirto di Panarea,120.0,USD
4,2218766,acqua di parma,Perfume,Colonia Miniature Set,72.0,USD


In [70]:
cosmetics_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8632 entries, 0 to 8631
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        8632 non-null   int64  
 1   brand     8632 non-null   object 
 2   category  8632 non-null   object 
 3   name      8632 non-null   object 
 4   price     8632 non-null   float64
 5   currency  8632 non-null   object 
dtypes: float64(1), int64(1), object(4)
memory usage: 404.8+ KB


In [71]:
ingredient_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8632 entries, 0 to 8631
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   product_id   8632 non-null   int64 
 1   ingredients  8632 non-null   object
dtypes: int64(1), object(1)
memory usage: 135.0+ KB


In [72]:
category_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   sub_category    17 non-null     object
 1   broad_category  17 non-null     object
dtypes: object(2)
memory usage: 400.0+ bytes
